In [1]:
from constants import (AZURE_OPENAI_ENDPOINT,
                       AZURE_OPENAI_API_KEY,
                       API_VERSION,
                       LLM,
                       OPENAI_API_KEY
                       )


In [2]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

## LLM

In [3]:
llm = AzureOpenAI(
    model=LLM,
    deployment_name="proxy_llm",
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=API_VERSION,
)

## EMBEDDING MODEL

In [15]:
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",
    deployment_name="proxy_embedding_model",
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=API_VERSION,
)

In [16]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

# ETL

In [21]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

import os

In [22]:
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [23]:
# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    required_exts = [".txt"]
    reader = SimpleDirectoryReader(input_dir="/workspaces/Intranet_Innovation_Nx3tCoder/data/raw/v2_unzipped/txt/",
                                  required_exts=required_exts,
                                  recursive=True)
    documents = reader.load_data()
    print(f"Loaded {len(documents)} docs")
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)


Loaded 44 docs


Retrying llama_index.embeddings.openai.base.get_embeddings in 0.2033173556224953 seconds as it raised APIConnectionError: Connection error..


APIConnectionError: Connection error.

In [18]:
type(documents)

list

In [19]:
documents[0]

Document(id_='1f2299ee-0349-4c24-8c40-6b1c3c15a220', embedding=None, metadata={'file_path': '/workspaces/Intranet_Innovation_Nx3tCoder/data/raw/v2_unzipped/txt/Büro Bürgermeister/Da01-02_Presse.txt', 'file_name': 'Da01-02_Presse.txt', 'file_type': 'text/plain', 'file_size': 4398, 'creation_date': '2024-05-10', 'last_modified_date': '2024-04-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=" \nGuidelines for Press and Public Relations of the City Administration of Fictitiousville\nThe press and public relations of the City Administration of Fictitiousville primarily aims to publicize the services provided by all organizational units as shown in the Departmental Distribution Plan. The goal is to create a positive image of the entire administration. Pre

In [17]:
index = VectorStoreIndex.from_documents(documents)

Retrying llama_index.embeddings.openai.base.get_embeddings in 0.3030753171172802 seconds as it raised BadRequestError: Error code: 400 - {'error': {'code': 400, 'message': "The request model 'proxy_embedding_model' is not available for this event. Available models are: contoso-products, gpt-35-turbo, gpt-4, text-embedding-3-small, text-embedding-ada-002"}}.
Retrying llama_index.embeddings.openai.base.get_embeddings in 1.2769758278659078 seconds as it raised BadRequestError: Error code: 400 - {'error': {'code': 400, 'message': "The request model 'proxy_embedding_model' is not available for this event. Available models are: contoso-products, gpt-35-turbo, gpt-4, text-embedding-3-small, text-embedding-ada-002"}}.
Retrying llama_index.embeddings.openai.base.get_embeddings in 2.1919445616573174 seconds as it raised BadRequestError: Error code: 400 - {'error': {'code': 400, 'message': "The request model 'proxy_embedding_model' is not available for this event. Available models are: contoso-pr

BadRequestError: Error code: 400 - {'error': {'code': 400, 'message': "The request model 'proxy_embedding_model' is not available for this event. Available models are: contoso-products, gpt-35-turbo, gpt-4, text-embedding-3-small, text-embedding-ada-002"}}

In [4]:
client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=API_VERSION,
)

In [5]:
MESSAGES = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020.",
    },
    {"role": "user", "content": "Where was it played?"},
]

In [6]:
completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=MESSAGES,
)

print(completion.model_dump_json(indent=2))

{
  "id": "chatcmpl-9NQfP1mXvmnCQQOP1YRnQ7lO6EVo7",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The 2020 World Series was played at Globe Life Field in Arlington, Texas.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1715371079,
  "model": "gpt-35-turbo:swedencentral",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 53,
    "total_tokens": 70
  },